# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,sainte-anne-des-monts,49.1240,-66.4924,18.93,63,{'all': 100},1.90,CA,1691506800
1,queenstown,-31.8976,26.8753,17.80,38,{'all': 85},3.15,ZA,1691506800
2,hermanus,-34.4187,19.2345,16.56,71,{'all': 46},5.64,ZA,1691506800
3,dryden,49.7833,-92.7503,20.03,94,{'all': 40},1.88,CA,1691506800
4,sandnessjoen,66.0217,12.6316,27.06,50,{'all': 20},7.18,NO,1691506800


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City"
)

# Display the map
humid_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria: temp between 20 and 25, humidity less than 50%, wind speed less than 5
target_cities = city_data_df.loc[(city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <=25) & 
                                 (city_data_df["Humidity"] <50) &
                                 (city_data_df["Wind Speed"] < 5)
                                ]
target_cities = target_cities.reset_index(drop=True)

# Drop any rows with null values
target_cities = target_cities.dropna()

# Display sample data
target_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mpika,-11.8343,31.4529,22.29,30,{'all': 15},3.30,ZM,1691506800
1,chu,43.5983,73.7614,24.33,47,{'all': 64},0.97,KZ,1691506800
2,klyuchi,52.2667,79.1667,21.45,42,{'all': 34},1.47,RU,1691506800
3,yurgamysh,55.3714,64.4656,22.50,49,{'all': 99},2.20,RU,1691506800
4,udachny,66.4167,112.4000,21.92,43,{'all': 41},2.90,RU,1691506800
5,kievka,50.2619,71.5514,23.89,45,{'all': 81},4.36,KZ,1691506800
6,bayanaul,50.7889,75.6956,21.79,34,{'all': 63},3.79,KZ,1691506800
7,tshabong,-26.0500,22.4500,24.86,10,{'all': 1},2.50,BW,1691506800
8,novodvinsk,64.4165,40.8122,23.76,48,{'all': 0},3.11,RU,1691506800
9,mahalapye,-23.1041,26.8142,23.57,13,{'all': 0},3.86,BW,1691506800


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = target_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mpika,ZM,-11.8343,31.4529,30,
1,chu,KZ,43.5983,73.7614,47,
2,klyuchi,RU,52.2667,79.1667,42,
3,yurgamysh,RU,55.3714,64.4656,49,
4,udachny,RU,66.4167,112.4000,43,
5,kievka,KZ,50.2619,71.5514,45,
6,bayanaul,KZ,50.7889,75.6956,34,
7,tshabong,BW,-26.0500,22.4500,10,
8,novodvinsk,RU,64.4165,40.8122,48,
9,mahalapye,BW,-23.1041,26.8142,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10_000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mpika - nearest hotel: Melody Lodge
chu - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
yurgamysh - nearest hotel: Уют
udachny - nearest hotel: Вилюй
kievka - nearest hotel: No hotel found
bayanaul - nearest hotel: Жан
tshabong - nearest hotel: No hotel found
novodvinsk - nearest hotel: Уют
mahalapye - nearest hotel: Lme Lodge
altai - nearest hotel: Altai hotel 40000-100000T
cedar city - nearest hotel: Baymont Inn & Suites
horezu - nearest hotel: Hotel Horezu
meadow lake - nearest hotel: No hotel found
suntar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mpika,ZM,-11.8343,31.4529,30,Melody Lodge
1,chu,KZ,43.5983,73.7614,47,No hotel found
2,klyuchi,RU,52.2667,79.1667,42,No hotel found
3,yurgamysh,RU,55.3714,64.4656,49,Уют
4,udachny,RU,66.4167,112.4000,43,Вилюй
5,kievka,KZ,50.2619,71.5514,45,No hotel found
6,bayanaul,KZ,50.7889,75.6956,34,Жан
7,tshabong,BW,-26.0500,22.4500,10,No hotel found
8,novodvinsk,RU,64.4165,40.8122,48,Уют
9,mahalapye,BW,-23.1041,26.8142,13,Lme Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 64,
    color="City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)